In [83]:
import re
import ast
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st

import nltk 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


import warnings
warnings.filterwarnings('ignore')


In [84]:
movie= pd.read_csv('IMDB-Movie-Dataset(2023-1951).csv')


In [85]:
movie.head(2)

,movie_id,movie_name,year,genre,overview,director,cast
0,tt15354916,Jawan,2023,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ..."
1,tt15748830,Jaane Jaan,2023,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ..."


In [86]:
movie.shape

(2199, 7)

In [87]:
movie.columns

Index(['movie_id', 'movie_name', 'year', 'genre', 'overview', 'director',
       'cast'],
      dtype='object')

# choose the Relevent Features for Movies Recommendation 
- genres
- Title
- overviews
- movie_id
- cast
- director

In [88]:
movie[:4]

,movie_id,movie_name,year,genre,overview,director,cast
0,tt15354916,Jawan,2023,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ..."
1,tt15748830,Jaane Jaan,2023,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ..."
2,tt11663228,Jailer,2023,"Action, Comedy, Crime",A retired jailer goes on a manhunt to find his...,Nelson Dilipkumar,"Rajinikanth, Mohanlal, Shivarajkumar, Jackie S..."
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,2023,"Comedy, Drama, Family",Flamboyant Punjabi Rocky and intellectual Beng...,Karan Johar,"Ranveer Singh, Alia Bhatt, Dharmendra, Shabana..."


In [89]:
df = movie[['movie_id' , 'movie_name' ,'genre','overview' , 'director' , 'cast']]
df.head(4)

,movie_id,movie_name,genre,overview,director,cast
0,tt15354916,Jawan,"Action, Thriller",A high-octane action thriller which outlines t...,Atlee,"Shah Rukh Khan, Nayanthara, Vijay Sethupathi, ..."
1,tt15748830,Jaane Jaan,"Crime, Drama, Mystery",A single mother and her daughter who commit a ...,Sujoy Ghosh,"Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, ..."
2,tt11663228,Jailer,"Action, Comedy, Crime",A retired jailer goes on a manhunt to find his...,Nelson Dilipkumar,"Rajinikanth, Mohanlal, Shivarajkumar, Jackie S..."
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,"Comedy, Drama, Family",Flamboyant Punjabi Rocky and intellectual Beng...,Karan Johar,"Ranveer Singh, Alia Bhatt, Dharmendra, Shabana..."


In [90]:
df.shape

(2199, 6)

Final_Goals = 'title' + 'Movie_id'  + 'Tags'     #tags means it contain all important data of movies which is get by overviews , crew ,cast , keywords.

In [91]:
df.isnull().sum()

movie_id      0
movie_name    0
genre         0
overview      0
director      0
cast          0
dtype: int64

In [92]:
df['overview']

0       A high-octane action thriller which outlines t...
1       A single mother and her daughter who commit a ...
2       A retired jailer goes on a manhunt to find his...
3       Flamboyant Punjabi Rocky and intellectual Beng...
4       An unhappy civilian asks the court to mandate ...
                              ...                        
2194                                           Add a Plot
2195    A renowned music teacher mentors a promising y...
2196    When a ballroom dancer's shot at a crucial tou...
2197    After the tragic deaths of his son Ajit and da...
2198    Raj is a successful lawyer due to constant che...
Name: overview, Length: 2199, dtype: object

In [93]:
df['genre'] = df['genre'].apply(lambda x : x.split())


In [94]:
df['overview'][0]

'A high-octane action thriller which outlines the emotional journey of a man who is set to rectify the wrongs in the society.'

In [95]:
df['genre']

0               [Action,, Thriller]
1         [Crime,, Drama,, Mystery]
2         [Action,, Comedy,, Crime]
3         [Comedy,, Drama,, Family]
4                  [Comedy,, Drama]
                   ...             
2194                     [Thriller]
2195    [Drama,, Musical,, Romance]
2196            [Musical,, Romance]
2197     [Drama,, Family,, Fantasy]
2198      [Action,, Comedy,, Drama]
Name: genre, Length: 2199, dtype: object

In [96]:
df['genre'][0]

['Action,', 'Thriller']

In [97]:
df['director'][0]

'Atlee'

In [98]:
df['director'] = df['director'].apply(lambda x : x.split())


In [99]:
df['cast'][0]

'Shah Rukh Khan, Nayanthara, Vijay Sethupathi, Deepika Padukone'

In [100]:
df['cast'] = df['cast'].apply(lambda x : x.split())


# Lets start data Cleaning

- Data is Already Clean , So we Prepare Model for Recommendation|

In [101]:
df.head(3)

,movie_id,movie_name,genre,overview,director,cast
0,tt15354916,Jawan,"[Action,, Thriller]",A high-octane action thriller which outlines t...,[Atlee],"[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup..."
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]",A single mother and her daughter who commit a ...,"[Sujoy, Ghosh]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V..."
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]",A retired jailer goes on a manhunt to find his...,"[Nelson, Dilipkumar]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack..."


### dekho bhai humlog  ko wahi  data lena joh serach kare toh usse movie recommendation ho.koi editor ka name likh kar movie ka name nhi bta sakta toh kyu data ko complex karu.joh important data hai usko hi lena hai.

- Overviews

In [102]:
'I am Bad Boy'.split()    # Exmaple Of splits

['I', 'am', 'Bad', 'Boy']

In [103]:
df.isnull().sum()

movie_id      0
movie_name    0
genre         0
overview      0
director      0
cast          0
dtype: int64

In [104]:
df.dropna(inplace=True)

In [105]:
df.isnull().sum()

movie_id      0
movie_name    0
genre         0
overview      0
director      0
cast          0
dtype: int64

In [106]:
df['overview'] = df['overview'].apply(lambda x : x.split())

In [107]:
df[: 3]

,movie_id,movie_name,genre,overview,director,cast
0,tt15354916,Jawan,"[Action,, Thriller]","[A, high-octane, action, thriller, which, outl...",[Atlee],"[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup..."
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]","[A, single, mother, and, her, daughter, who, c...","[Sujoy, Ghosh]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V..."
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]","[A, retired, jailer, goes, on, a, manhunt, to,...","[Nelson, Dilipkumar]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack..."


In [108]:
df['Tags'] = df['overview'] + df['genre'] + df['director'] + df['cast']

In [109]:
df.head(3)

,movie_id,movie_name,genre,overview,director,cast,Tags
0,tt15354916,Jawan,"[Action,, Thriller]","[A, high-octane, action, thriller, which, outl...",[Atlee],"[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup...","[A, high-octane, action, thriller, which, outl..."
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]","[A, single, mother, and, her, daughter, who, c...","[Sujoy, Ghosh]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V...","[A, single, mother, and, her, daughter, who, c..."
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]","[A, retired, jailer, goes, on, a, manhunt, to,...","[Nelson, Dilipkumar]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack...","[A, retired, jailer, goes, on, a, manhunt, to,..."


# Final DataFrame of Data

In [110]:
data = df[['movie_id' , 'movie_name' ,'genre','cast', 'Tags']]
data

,movie_id,movie_name,genre,cast,Tags
0,tt15354916,Jawan,"[Action,, Thriller]","[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup...","[A, high-octane, action, thriller, which, outl..."
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V...","[A, single, mother, and, her, daughter, who, c..."
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack...","[A, retired, jailer, goes, on, a, manhunt, to,..."
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,"[Comedy,, Drama,, Family]","[Ranveer, Singh,, Alia, Bhatt,, Dharmendra,, S...","[Flamboyant, Punjabi, Rocky, and, intellectual..."
4,tt15732324,OMG 2,"[Comedy,, Drama]","[Pankaj, Tripathi,, Akshay, Kumar,, Yami, Gaut...","[An, unhappy, civilian, asks, the, court, to, ..."
...,...,...,...,...,...
2194,tt11112474,Heeriye,[Thriller],"[Shatrughan, Sinha,, Reena, Roy,, Ajit, Khan,,...","[Add, a, Plot, Thriller, Subhash, Ghai, Shatru..."
2195,tt0332766,Sur: The Melody of Life,"[Drama,, Musical,, Romance]","[Lucky, Ali,, Simone, Singh,, Achint, Kaur,, E...","[A, renowned, music, teacher, mentors, a, prom..."
2196,tt8622232,Time to Dance,"[Musical,, Romance]","[Sooraj, Pancholi,, Isabelle, Kaif,, Waluscha,...","[When, a, ballroom, dancer's, shot, at, a, cru..."
2197,tt0187351,Nigahen: Nagina Part II,"[Drama,, Family,, Fantasy]","[Sunny, Deol,, Sridevi,, Anupam, Kher,, Gulsha...","[After, the, tragic, deaths, of, his, son, Aji..."


 - Next Step to Remove the space present in between the words.then again convert it into a String.

In [111]:
'i am a boy'.replace(' ','')     # for Example

'iamaboy'

In [112]:
print(data['Tags'][0])

['A', 'high-octane', 'action', 'thriller', 'which', 'outlines', 'the', 'emotional', 'journey', 'of', 'a', 'man', 'who', 'is', 'set', 'to', 'rectify', 'the', 'wrongs', 'in', 'the', 'society.', 'Action,', 'Thriller', 'Atlee', 'Shah', 'Rukh', 'Khan,', 'Nayanthara,', 'Vijay', 'Sethupathi,', 'Deepika', 'Padukone']


In [113]:
data['Tags'] = data['Tags'].apply(lambda x : [i.replace(' ' ,'') for i in x])

In [114]:
print(data['Tags'][0])


['A', 'high-octane', 'action', 'thriller', 'which', 'outlines', 'the', 'emotional', 'journey', 'of', 'a', 'man', 'who', 'is', 'set', 'to', 'rectify', 'the', 'wrongs', 'in', 'the', 'society.', 'Action,', 'Thriller', 'Atlee', 'Shah', 'Rukh', 'Khan,', 'Nayanthara,', 'Vijay', 'Sethupathi,', 'Deepika', 'Padukone']


In [115]:
data[:3]

,movie_id,movie_name,genre,cast,Tags
0,tt15354916,Jawan,"[Action,, Thriller]","[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup...","[A, high-octane, action, thriller, which, outl..."
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V...","[A, single, mother, and, her, daughter, who, c..."
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack...","[A, retired, jailer, goes, on, a, manhunt, to,..."


- convert into string 

In [116]:
''.join(['i' , 'am', 'a','boy'])       # Example 

'iamaboy'

In [117]:
' '.join(['i' , 'am', 'a','boy'])     # Example

'i am a boy'

In [118]:
data['Tags'] = data['Tags'].apply(lambda x : ' '.join(x))

In [119]:
data[:3]

,movie_id,movie_name,genre,cast,Tags
0,tt15354916,Jawan,"[Action,, Thriller]","[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup...",A high-octane action thriller which outlines t...
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V...",A single mother and her daughter who commit a ...
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack...",A retired jailer goes on a manhunt to find his...


In [120]:
data['Tags'][1]

'A single mother and her daughter who commit a crime and a neighbor who helps them cover it up amidst a police investigation. Crime, Drama, Mystery Sujoy Ghosh Kareena Kapoor, Jaideep Ahlawat, Vijay Varma, Saurabh Sachdeva'

# NLP(Natural Language Processing)  Concept to Preprocess Textual Data

- Lower case
- Tokenization-------------> it means that it split every word into single variable(shiek ==  's','h','i','e','k')
- stopwords Removal -----> it remove the word like (i ,am ,is, etc)
- stemming

# TEST PREPROCESS FUNCTION

In [121]:
ps = PorterStemmer()

def preprocess_text(text):
    
    new_text = []
    
    for i in text.split():
        lower = i.lower()
        new_text.append(ps.stem(lower))

    return " ".join(new_text)

In [122]:
preprocess_text(data['Tags'][0])

'a high-octan action thriller which outlin the emot journey of a man who is set to rectifi the wrong in the society. action, thriller atle shah rukh khan, nayanthara, vijay sethupathi, deepika padukon'

In [123]:
data['Tags'].apply(preprocess_text)

0       a high-octan action thriller which outlin the ...
1       a singl mother and her daughter who commit a c...
2       a retir jailer goe on a manhunt to find hi son...
3       flamboy punjabi rocki and intellectu bengali j...
4       an unhappi civilian ask the court to mandat co...
                              ...                        
2194    add a plot thriller subhash ghai shatrughan si...
2195    a renown music teacher mentor a promis young s...
2196    when a ballroom dancer' shot at a crucial tour...
2197    after the tragic death of hi son ajit and daug...
2198    raj is a success lawyer due to constant cheat ...
Name: Tags, Length: 2199, dtype: object

In [124]:
data['Tags'] = data['Tags'].apply(preprocess_text)

In [125]:
data

,movie_id,movie_name,genre,cast,Tags
0,tt15354916,Jawan,"[Action,, Thriller]","[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup...",a high-octan action thriller which outlin the ...
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V...",a singl mother and her daughter who commit a c...
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack...",a retir jailer goe on a manhunt to find hi son...
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,"[Comedy,, Drama,, Family]","[Ranveer, Singh,, Alia, Bhatt,, Dharmendra,, S...",flamboy punjabi rocki and intellectu bengali j...
4,tt15732324,OMG 2,"[Comedy,, Drama]","[Pankaj, Tripathi,, Akshay, Kumar,, Yami, Gaut...",an unhappi civilian ask the court to mandat co...
...,...,...,...,...,...
2194,tt11112474,Heeriye,[Thriller],"[Shatrughan, Sinha,, Reena, Roy,, Ajit, Khan,,...",add a plot thriller subhash ghai shatrughan si...
2195,tt0332766,Sur: The Melody of Life,"[Drama,, Musical,, Romance]","[Lucky, Ali,, Simone, Singh,, Achint, Kaur,, E...",a renown music teacher mentor a promis young s...
2196,tt8622232,Time to Dance,"[Musical,, Romance]","[Sooraj, Pancholi,, Isabelle, Kaif,, Waluscha,...",when a ballroom dancer' shot at a crucial tour...
2197,tt0187351,Nigahen: Nagina Part II,"[Drama,, Family,, Fantasy]","[Sunny, Deol,, Sridevi,, Anupam, Kher,, Gulsha...",after the tragic death of hi son ajit and daug...


# BOW (BAGS OF WORDS)

In [126]:
cv = CountVectorizer(max_features=5000, stop_words='english')
cv

CountVectorizer(max_features=5000, stop_words='english')

In [127]:
vectors = cv.fit_transform(data['Tags'])
vectors

<2199x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 52009 stored elements in Compressed Sparse Row format>

In [128]:
vectors = cv.fit_transform(data['Tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [129]:
len(vectors[0]), vectors[0]

(5000, array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [130]:
for i in cv.get_feature_names_out():
    print(i)

000
10
100
11
12
13
14
15
16
17
18
1857
19
1947
1960s
1962
1970
1970s
1971
1975
1980
1980s
1984
1990
1990s
1993
1994
1999
19th
20
2002
2003
2012
2013
2015
2016
2018
2020
21
21st
23
24
25
26
27
2nd
30
300
36
40
48
50
500
55
60
80
90
aadarsh
aahana
aakash
aakeel
aamir
aanand
aarti
aaryan
aatish
aayush
abandon
abba
abduct
abduction
abdul
abhay
abhilash
abhilasha
abhimanu
abhimanyu
abhinav
abhinay
abhishek
abhyankar
abid
abil
abl
abraham
abrar
abroad
abu
abudhar
abus
accept
accid
accident
accompani
accomplish
account
accus
acharya
achiev
achint
achyuth
acid
acp
acquaint
acquir
act
action
actions
activ
activist
activities
actor
actress
actual
adah
adajania
adapt
adarsh
add
addict
adhikari
adi
adil
aditi
aditya
adiya
adnan
adopt
adult
adulteri
adults
advait
advani
adventur
adventure
advertis
adyasha
aerial
affair
affect
affluent
afford
afghan
afghanistan
aftab
afzal
ag
agarwal
agashe
agastya
age
agenc
agency
agenda
agent
ages
aggarw
aggarwal
agha
agnihotri
ago
agraw
agre
agrohari
aham
ahlawa

In [131]:
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.04003204, 0.08333333, ..., 0.        , 0.        ,
        0.06299408],
       [0.04003204, 1.        , 0.08006408, ..., 0.        , 0.06362848,
        0.03026138],
       [0.08333333, 0.08006408, 1.        , ..., 0.04166667, 0.09933993,
        0.18898224],
       ...,
       [0.        , 0.        , 0.04166667, ..., 1.        , 0.06622662,
        0.09449112],
       [0.        , 0.06362848, 0.09933993, ..., 0.06622662, 1.        ,
        0.25031309],
       [0.06299408, 0.03026138, 0.18898224, ..., 0.09449112, 0.25031309,
        1.        ]])

In [132]:
similarity[0]

array([1.        , 0.04003204, 0.08333333, ..., 0.        , 0.        ,
       0.06299408])

In [133]:
sorted(similarity[0])

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [134]:
sorted(similarity[0])[-10:]

[0.29166666666666674,
 0.297939785765562,
 0.29814239699997197,
 0.31180478223116187,
 0.31180478223116187,
 0.32274861218395146,
 0.3265986323710905,
 0.33557802760701216,
 0.33678765702728175,
 0.9999999999999999]

In [135]:
sorted(similarity[0], reverse=True)

[0.9999999999999999,
 0.33678765702728175,
 0.33557802760701216,
 0.3265986323710905,
 0.32274861218395146,
 0.31180478223116187,
 0.31180478223116187,
 0.29814239699997197,
 0.297939785765562,
 0.29166666666666674,
 0.29166666666666674,
 0.2886751345948129,
 0.2886751345948129,
 0.2886751345948129,
 0.28426762180748066,
 0.28022426915890253,
 0.28022426915890253,
 0.280056016805602,
 0.28005601680560194,
 0.2738612787525831,
 0.2738612787525831,
 0.26533431525261625,
 0.2653343152526162,
 0.2649064714130088,
 0.26111648393354675,
 0.2608745973749755,
 0.25819888974716115,
 0.25663245128736834,
 0.2553769592276246,
 0.2551551815399144,
 0.25000000000000006,
 0.25000000000000006,
 0.2475368857441686,
 0.2461829819586655,
 0.2461829819586655,
 0.241522945769824,
 0.24056261216234412,
 0.24056261216234412,
 0.24019223070763074,
 0.2381448361039201,
 0.2357022603955159,
 0.2357022603955159,
 0.23414645289542352,
 0.23145502494313788,
 0.22742941307367107,
 0.22645540682891918,
 0.223606797

In [136]:
list(enumerate(similarity[0]))

[(0, 0.9999999999999999),
 (1, 0.04003203845127179),
 (2, 0.08333333333333336),
 (3, 0.0),
 (4, 0.0),
 (5, 0.04082482904638631),
 (6, 0.07856742013183862),
 (7, 0.04082482904638631),
 (8, 0.3265986323710905),
 (9, 0.1342312110428049),
 (10, 0.03857583749052298),
 (11, 0.04003203845127179),
 (12, 0.037904902178945175),
 (13, 0.0936585811581694),
 (14, 0.03666177875533833),
 (15, 0.0468292905790847),
 (16, 0.06454972243679029),
 (17, 0.11572751247156894),
 (18, 0.14433756729740646),
 (19, 0.11572751247156894),
 (20, 0.2475368857441686),
 (21, 0.0),
 (22, 0.11785113019775793),
 (23, 0.03928371006591931),
 (24, 0.1276884796138123),
 (25, 0.13693063937629155),
 (26, 0.16012815380508716),
 (27, 0.07332355751067667),
 (28, 0.059549133417541374),
 (29, 0.0),
 (30, 0.10660035817780525),
 (31, 0.08512565307587487),
 (32, 0.0),
 (33, 0.32274861218395146),
 (34, 0.0),
 (35, 0.04003203845127179),
 (36, 0.0),
 (37, 0.0331133089266261),
 (38, 0.21281413268968719),
 (39, 0.26111648393354675),
 (40, 0.

In [137]:
sorted(list(enumerate(similarity[0])), reverse=True)

[(2198, 0.06299407883487121),
 (2197, 0.0),
 (2196, 0.0),
 (2195, 0.04082482904638631),
 (2194, 0.16984155512168939),
 (2193, 0.12500000000000003),
 (2192, 0.0),
 (2191, 0.14907119849998599),
 (2190, 0.0),
 (2189, 0.07580980435789035),
 (2188, 0.0),
 (2187, 0.0),
 (2186, 0.07216878364870323),
 (2185, 0.04003203845127179),
 (2184, 0.0),
 (2183, 0.0),
 (2182, 0.0),
 (2181, 0.07453559924999299),
 (2180, 0.0),
 (2179, 0.08006407690254358),
 (2178, 0.0),
 (2177, 0.07453559924999299),
 (2176, 0.0),
 (2175, 0.0),
 (2174, 0.08164965809277262),
 (2173, 0.0),
 (2172, 0.08512565307587487),
 (2171, 0.0),
 (2170, 0.07715167498104596),
 (2169, 0.0468292905790847),
 (2168, 0.1404878717372541),
 (2167, 0.0),
 (2166, 0.11572751247156894),
 (2165, 0.0),
 (2164, 0.17010345435994292),
 (2163, 0.10910894511799621),
 (2162, 0.0),
 (2161, 0.2886751345948129),
 (2160, 0.10350983390135314),
 (2159, 0.0),
 (2158, 0.0),
 (2157, 0.09901475429766744),
 (2156, 0.03553345272593508),
 (2155, 0.06454972243679029),
 (2

In [138]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6]

[(1155, 0.33678765702728175),
 (76, 0.33557802760701216),
 (8, 0.3265986323710905),
 (33, 0.32274861218395146),
 (51, 0.31180478223116187)]

In [139]:
data['Tags'][0]

'a high-octan action thriller which outlin the emot journey of a man who is set to rectifi the wrong in the society. action, thriller atle shah rukh khan, nayanthara, vijay sethupathi, deepika padukon'

In [140]:
data.iloc[1216]

movie_id                                              tt0116308
movie_name                                                 Fire
genre                                         [Drama,, Romance]
cast          [Shabana, Azmi,, Nandita, Das,, Karishma, Jhal...
Tags          two women who are abandon by their husband fin...
Name: 1216, dtype: object

In [141]:
data.iloc[220]

movie_id                                              tt3767372
movie_name                                                 Piku
genre                                          [Comedy,, Drama]
cast          [Deepika, Padukone,, Amitabh, Bachchan,, Irrfa...
Tags          a quirki comedi about the relationship between...
Name: 220, dtype: object

In [142]:
data

,movie_id,movie_name,genre,cast,Tags
0,tt15354916,Jawan,"[Action,, Thriller]","[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup...",a high-octan action thriller which outlin the ...
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V...",a singl mother and her daughter who commit a c...
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack...",a retir jailer goe on a manhunt to find hi son...
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,"[Comedy,, Drama,, Family]","[Ranveer, Singh,, Alia, Bhatt,, Dharmendra,, S...",flamboy punjabi rocki and intellectu bengali j...
4,tt15732324,OMG 2,"[Comedy,, Drama]","[Pankaj, Tripathi,, Akshay, Kumar,, Yami, Gaut...",an unhappi civilian ask the court to mandat co...
...,...,...,...,...,...
2194,tt11112474,Heeriye,[Thriller],"[Shatrughan, Sinha,, Reena, Roy,, Ajit, Khan,,...",add a plot thriller subhash ghai shatrughan si...
2195,tt0332766,Sur: The Melody of Life,"[Drama,, Musical,, Romance]","[Lucky, Ali,, Simone, Singh,, Achint, Kaur,, E...",a renown music teacher mentor a promis young s...
2196,tt8622232,Time to Dance,"[Musical,, Romance]","[Sooraj, Pancholi,, Isabelle, Kaif,, Waluscha,...",when a ballroom dancer' shot at a crucial tour...
2197,tt0187351,Nigahen: Nagina Part II,"[Drama,, Family,, Fantasy]","[Sunny, Deol,, Sridevi,, Anupam, Kher,, Gulsha...",after the tragic death of hi son ajit and daug...


In [143]:
sorted(list(enumerate(similarity[3])), reverse=True, key=lambda x: x[1])[1:6]

[(2125, 0.3703280399090206),
 (81, 0.3518657752744984),
 (194, 0.3450327796711771),
 (2077, 0.31720831958260864),
 (383, 0.301624673408895)]

In [144]:
data.iloc[65]

movie_id                                             tt15073166
movie_name                                                IB 71
genre                                       [Action,, Thriller]
cast          [Anupam, Kher,, Vidyut, Jammwal,, Dalip, Tahil...
Tags          a two- front war between indian intellig agenc...
Name: 65, dtype: object

In [145]:
data.iloc[428]

movie_id                                              tt7778680
movie_name                                           Photograph
genre                                         [Drama,, Romance]
cast          [Nawazuddin, Siddiqui,, Sanya, Malhotra,, Sach...
Tags          a struggl street photograph in mumbai, pressur...
Name: 428, dtype: object

# FINAL FUNCTION

In [146]:
def recommend(movie):
    
    movie_index = data[data['movie_name'] == movie].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:4]
    
    for i in movie_list:
#         print(i[0])
        print(data.iloc[i[0]].movie_name)

In [147]:
def recommends(movie):
    
    movie_index = data[data['genre'] == movie].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:4]
    
    for i in movie_list:
#         print(i[0])
        print(data.iloc[i[0]].movie_name)

In [148]:
recommend('Jawan')

Martin
Don
Pathaan


In [149]:
filtered_movies = data[data["genre"].apply(lambda x: "Action" in x)]
filtered_movies

,movie_id,movie_name,genre,cast,Tags
683,tt28782545,The Purvanchal Files,[Action],"[R, ., Siddharth,, Shivani, Thakur,, Zarina, W...",a cop stori of reveng and how he tri to purifi...
821,tt7790918,Untitled,[Action],"[Kiara, Advani,, Nikitin, Dheer,, Anupam, Kher...","in the dark underbelli of the city, bloodsh bh..."
856,tt9592496,Pogaru,[Action],"[Dhruva, Sarja,, Rashmika, Mandanna,, Sampath,...","shiva, a local ruffian is unstopp in hi path a..."
1049,tt0068938,Mere Jeevan Saathi,"[Drama,, Adventure,, Action]","[Rajesh, Khanna,, Tanuja, Samarth,, Sujit, Kum...",a talent artist (rajesh khanna) fall in love w...
1114,tt21357600,Bombay,[Action],"[Gavie, Chahal,, Deepshikha, Nagpal,, Danish, ...","ula mhatre, an underworld don oper activ with ..."
1127,tt3807778,Badla Sherni Ka,[Action],"[Sapna, Sappu,, Amit, Pachori,, Junior, Johnny...",jungl woman sherni object when greedi hunter i...
1145,tt28539592,Rambo,[Action],"[Tiger, Shroff,, Janhvi, Kapoor]","add a plot action rohit dhawan tiger shroff, j..."
1221,tt21074588,Baap,[Action],"[Lankesh, Bhardwaj,, Sanjay, Dutt,, Sunny, Deo...",add a plot action vivek chauhan lankesh bhardw...
1225,tt0102701,Prahaar: The Final Attack,"[Drama,, Thriller,, Action]","[Nana, Patekar,, Dimple, Kapadia,, Gautam, Jog...","prahaar is the stori of major chauhan, a tough..."
1236,tt7963804,Pantham,[Action],"[Gopichand,, Mehreen, Pirzada,, Ajay,, Joy, Ba...",vikranth is a vigilant who ha target 'nayak bh...


In [150]:
data['movie_name'].values

array(['Jawan', 'Jaane Jaan', 'Jailer', ..., 'Time to Dance',
       'Nigahen: Nagina Part II', 'Kyo Kii... Main Jhuth Nahin Bolta'],
      dtype=object)

In [151]:
data.iloc[5].movie_name

'Sukhee'

In [152]:
data['movie_name'] 

0                                   Jawan
1                              Jaane Jaan
2                                  Jailer
3         Rocky Aur Rani Kii Prem Kahaani
4                                   OMG 2
                      ...                
2194                              Heeriye
2195              Sur: The Melody of Life
2196                        Time to Dance
2197              Nigahen: Nagina Part II
2198    Kyo Kii... Main Jhuth Nahin Bolta
Name: movie_name, Length: 2199, dtype: object

## Pickle Module

- To use pickle in Python, we must first create an object like my_object. Then, using the dump() function and the 'wb' parameter on open(), we can serialize this object to a file called my_object. pickle for future usage. This binary write mode ensures that our data is secure and stored correctly for later access.

In [153]:
data

,movie_id,movie_name,genre,cast,Tags
0,tt15354916,Jawan,"[Action,, Thriller]","[Shah, Rukh, Khan,, Nayanthara,, Vijay, Sethup...",a high-octan action thriller which outlin the ...
1,tt15748830,Jaane Jaan,"[Crime,, Drama,, Mystery]","[Kareena, Kapoor,, Jaideep, Ahlawat,, Vijay, V...",a singl mother and her daughter who commit a c...
2,tt11663228,Jailer,"[Action,, Comedy,, Crime]","[Rajinikanth,, Mohanlal,, Shivarajkumar,, Jack...",a retir jailer goe on a manhunt to find hi son...
3,tt14993250,Rocky Aur Rani Kii Prem Kahaani,"[Comedy,, Drama,, Family]","[Ranveer, Singh,, Alia, Bhatt,, Dharmendra,, S...",flamboy punjabi rocki and intellectu bengali j...
4,tt15732324,OMG 2,"[Comedy,, Drama]","[Pankaj, Tripathi,, Akshay, Kumar,, Yami, Gaut...",an unhappi civilian ask the court to mandat co...
...,...,...,...,...,...
2194,tt11112474,Heeriye,[Thriller],"[Shatrughan, Sinha,, Reena, Roy,, Ajit, Khan,,...",add a plot thriller subhash ghai shatrughan si...
2195,tt0332766,Sur: The Melody of Life,"[Drama,, Musical,, Romance]","[Lucky, Ali,, Simone, Singh,, Achint, Kaur,, E...",a renown music teacher mentor a promis young s...
2196,tt8622232,Time to Dance,"[Musical,, Romance]","[Sooraj, Pancholi,, Isabelle, Kaif,, Waluscha,...",when a ballroom dancer' shot at a crucial tour...
2197,tt0187351,Nigahen: Nagina Part II,"[Drama,, Family,, Fantasy]","[Sunny, Deol,, Sridevi,, Anupam, Kher,, Gulsha...",after the tragic death of hi son ajit and daug...


In [154]:
data.to_dict()

{'movie_id': {0: 'tt15354916',
  1: 'tt15748830',
  2: 'tt11663228',
  3: 'tt14993250',
  4: 'tt15732324',
  5: 'tt18266472',
  6: 'tt18561736',
  7: 'tt3691740',
  8: 'tt12844910',
  9: 'tt15464390',
  10: 'tt1187043',
  11: 'tt14914988',
  12: 'tt3741834',
  13: 'tt13131610',
  14: 'tt22188452',
  15: 'tt28568397',
  16: 'tt21908676',
  17: 'tt15441054',
  18: 'tt27542289',
  19: 'tt0454876',
  20: 'tt0249371',
  21: 'tt0838221',
  22: 'tt5074352',
  23: 'tt8239946',
  24: 'tt15428134',
  25: 'tt24485052',
  26: 'tt1954470',
  27: 'tt15433600',
  28: 'tt13751694',
  29: 'tt15891388',
  30: 'tt19755170',
  31: 'tt6277462',
  32: 'tt20913276',
  33: 'tt18411490',
  34: 'tt8672856',
  35: 'tt15501640',
  36: 'tt0083987',
  37: 'tt2338151',
  38: 'tt1188996',
  39: 'tt3405236',
  40: 'tt26324936',
  41: 'tt8108198',
  42: 'tt23036080',
  43: 'tt22086334',
  44: 'tt15380630',
  45: 'tt0986264',
  46: 'tt4430212',
  47: 'tt22297828',
  48: 'tt10083340',
  49: 'tt27012110',
  50: 'tt1524524

In [155]:
pickle.dump(data.to_dict(), open('movies_dict.pkl', mode='wb'))

In [156]:
pickle.dump(similarity, open('similarity.pkl', mode='wb'))

In [157]:
data['movie_name'].values

array(['Jawan', 'Jaane Jaan', 'Jailer', ..., 'Time to Dance',
       'Nigahen: Nagina Part II', 'Kyo Kii... Main Jhuth Nahin Bolta'],
      dtype=object)

In [158]:
list(data['movie_name'].values)

['Jawan',
 'Jaane Jaan',
 'Jailer',
 'Rocky Aur Rani Kii Prem Kahaani',
 'OMG 2',
 'Sukhee',
 'The Great Indian Family',
 'The BFG',
 'Pathaan',
 'Mastaney',
 '3 Idiots',
 'Satyaprem Ki Katha',
 'Lion',
 'Bholaa Shankar',
 'Haddi',
 'Friday Night Plan',
 'Neeyat',
 'Gadar 2',
 'Por Thozhil',
 'Life of Pi',
 'Asoka',
 'The Darjeeling Limited',
 'Dangal',
 'Tumbbad',
 'Dunki',
 'Sirf Ek Bandaa Kaafi Hai',
 'Gangs of Wasseypur',
 'Khufiya',
 'Animal',
 'Miss Shetty Mr Polishetty',
 'Bawaal',
 'Brahmastra Part One: Shiva',
 'Ghoomer',
 'Tiger 3',
 'Tu Jhoothi Main Makkaar',
 'Drishyam 2',
 'Gandhi',
 'PK',
 'My Name Is Khan',
 'Raees',
 'Rama Banam',
 'Andhadhun',
 'Thank You for Coming',
 'Dream Girl 2',
 'Kushi',
 'Taare Zameen Par',
 'Drishyam',
 'Chor Nikal Ke Bhaga',
 'Gangubai Kathiawadi',
 'Gaslight',
 'The Buckingham Murders',
 'Fighter',
 'Bholaa',
 'Om Shanti Om',
 'Kabhi Khushi Kabhie Gham...',
 'The Kerala Story',
 'Vikram Vedha',
 'Swades: We, the People',
 'Bajrangi Bhaijaan'